# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Rodrigo Martín del Campo

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://7239a1f7373c:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/25 19:57:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Build schema
# Import your module
from martindelcampo.spark_utils import SparkUtils
from pyspark.sql.functions import get_json_object, col

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
schema_cars      = SparkUtils.generate_schema([("car_id", "int"),    ("car_info", "string")])
schema_customers = SparkUtils.generate_schema([("customer_id", "int"),("customer_info", "string")])
schema_rentals   = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_cars     = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_customers= spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_rentals  = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")


# parse json columns and select only required columns
df_agencies  = df_agencies.withColumn("agency_name",   get_json_object(df_agencies.agency_info,   "$.agency_name")) \
                          .select(col("agency_id").cast("int").alias("agency_id"), "agency_name")

df_cars      = df_cars.withColumn("car_name",          get_json_object(df_cars.car_info,          "$.car_name")) \
                      .select(col("car_id").cast("int").alias("car_id"), "car_name")

df_customers = df_customers.withColumn("customer_name", get_json_object(df_customers.customer_info,"$.customer_name")) \
                           .select(col("customer_id").cast("int").alias("customer_id"), "customer_name")

df_rentals   = df_rentals \
    .withColumn("car_id",      get_json_object(df_rentals.rental_info, "$.car_id").cast("int")) \
    .withColumn("customer_id", get_json_object(df_rentals.rental_info, "$.customer_id").cast("int")) \
    .withColumn("agency_id",   get_json_object(df_rentals.rental_info, "$.agency_id").cast("int")) \
    .select(col("rental_id").cast("int").alias("rental_id"), "car_id", "customer_id", "agency_id")

# join dataframes
(
    df_rentals
    .join(df_cars,      "car_id",      "left")
    .join(df_agencies,  "agency_id",   "left")
    .join(df_customers, "customer_id", "left")
    .select("rental_id", "car_name", "agency_name", "customer_name")
    .orderBy("rental_id")

# display result with a single action at the end
    .show(truncate=False)   
)


+---------+------------------------------------+-------------+----------------+
|rental_id|car_name                            |agency_name  |customer_name   |
+---------+------------------------------------+-------------+----------------+
|0        |Grimes-Green Model 8                |SF Cars      |Laura Perry     |
|1        |Wallace-Carlson Model 9             |SF Cars      |Melanie Patrick |
|2        |Wagner LLC Model 1                  |NYC Rentals  |Theresa Estrada |
|3        |Bolton, Burns and Turner Model 10   |LA Car Rental|Amanda Garcia   |
|4        |Alvarez-Davis Model 3               |SF Cars      |Corey Cook      |
|5        |Grimes-Green Model 8                |SF Cars      |Kyle Ramos      |
|6        |Grimes-Green Model 8                |NYC Rentals  |Julie Chen      |
|7        |Clayton-Cook Model 10               |LA Car Rental|Sarah Peterson  |
|8        |Garcia, Hamilton and Carr Model 5   |SF Cars      |Jay Walsh       |
|9        |Chang-Fisher Model 7         

In [5]:
sc.stop()